In [3]:
import os
import json
import pandas as pd
import traceback



In [23]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
KEY=os.getenv("OPENAI_API_KEY")
print(KEY)



sk-proj-qHCkjW3wWxipabSH8RzCT3BlbkFJbQ0GQkSVSmOxrW4GyiHL


In [24]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [34]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [27]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [28]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [30]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [31]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [32]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [35]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [36]:
file_path="D:\Sachin Learning\Genrative AI\prj01\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()
print(TEXT)

Rahul Dravid, often referred to as "The Wall" of Indian cricket, is celebrated as one of the most technically proficient and mentally resilient batsmen in the history of the game. Born on January 11, 1973, in Indore, India, Dravid's cricketing journey is marked by his unwavering dedication, classical batting technique, and a career that spanned over a decade and a half, making him a legendary figure in the cricketing world.

Dravid made his Test debut for India on June 20, 1996, against England at Lord's. His debut innings of 95 runs showcased his immense potential and technical soundness. Over the years, he built a reputation as a reliable and consistent batsman, particularly in Test cricket. Known for his ability to anchor the innings, Dravid often played the role of a stabilizer in the Indian batting lineup, allowing more aggressive players to flourish around him.

One of the defining features of Dravid's career was his ability to adapt to different formats of the game. While he was

In [37]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [39]:
NUMBER=5 
SUBJECT="Cricket"
TONE="simple"

In [40]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Rahul Dravid, often referred to as "The Wall" of Indian cricket, is celebrated as one of the most technically proficient and mentally resilient batsmen in the history of the game. Born on January 11, 1973, in Indore, India, Dravid's cricketing journey is marked by his unwavering dedication, classical batting technique, and a career that spanned over a decade and a half, making him a legendary figure in the cricketing world.

Dravid made his Test debut for India on June 20, 1996, against England at Lord's. His debut innings of 95 runs showcased his immense potential and technical soundness. Over the years, he built a reputation as a reliable and consistent batsman, particularly in Test cricket. Known for his ability to anchor the innings, Dravid often played the role of a stabilizer in the Indian batting lineup, allowing more aggressive players to flourish around him.

One of the 

In [41]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2012
Prompt Tokens:1156
Completion Tokens:856
Total Cost:0.003446


In [44]:

quiz=response.get("quiz")
quiz=json.loads(quiz)
print(quiz)

{'1': {'mcq': 'What is Rahul Dravid often referred to as in Indian cricket?', 'options': {'a': 'The Rocket', 'b': 'The Wall', 'c': 'The Flash', 'd': 'The Beast'}, 'correct': 'b'}, '2': {'mcq': 'When did Rahul Dravid make his Test debut for India?', 'options': {'a': 'June 20, 1996', 'b': 'January 11, 1973', 'c': 'July 4, 2000', 'd': 'March 15, 1985'}, 'correct': 'a'}, '3': {'mcq': "What is one of the defining features of Dravid's career?", 'options': {'a': 'His aggressive batting style', 'b': 'His ability to bowl spin', 'c': 'His adaptability to different formats of the game', 'd': 'His fast bowling skills'}, 'correct': 'c'}, '4': {'mcq': 'In which cricket format did Dravid amass over 13,000 runs at an average of 52.63?', 'options': {'a': 'T20 cricket', 'b': 'One Day Internationals (ODIs)', 'c': 'Test cricket', 'd': 'County cricket'}, 'correct': 'c'}, '5': {'mcq': 'What role did Rahul Dravid take on during the early 2000s to help India field an extra batsman?', 'options': {'a': 'Opening

In [46]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
print(quiz_table_data)

[{'MCQ': 'What is Rahul Dravid often referred to as in Indian cricket?', 'Choices': 'a: The Rocket | b: The Wall | c: The Flash | d: The Beast', 'Correct': 'b'}, {'MCQ': 'When did Rahul Dravid make his Test debut for India?', 'Choices': 'a: June 20, 1996 | b: January 11, 1973 | c: July 4, 2000 | d: March 15, 1985', 'Correct': 'a'}, {'MCQ': "What is one of the defining features of Dravid's career?", 'Choices': 'a: His aggressive batting style | b: His ability to bowl spin | c: His adaptability to different formats of the game | d: His fast bowling skills', 'Correct': 'c'}, {'MCQ': 'In which cricket format did Dravid amass over 13,000 runs at an average of 52.63?', 'Choices': 'a: T20 cricket | b: One Day Internationals (ODIs) | c: Test cricket | d: County cricket', 'Correct': 'c'}, {'MCQ': 'What role did Rahul Dravid take on during the early 2000s to help India field an extra batsman?', 'Choices': 'a: Opening batsman | b: Wicketkeeper-batsman | c: Fast bowler | d: Leg spinner', 'Correct'

In [47]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)